In [2]:
import os
import colorcet as cc
from colour import Color

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

import pymeshlab as ml

import pyvista as pv
from pyvista import examples


from collections import defaultdict

from colorspacious import cspace_converter
pv.global_theme.transparent_background = True

## Put Multiple Figures in One Using Matplotlib

### Get all the PNG files

In [5]:
pialfiles = []
whitefiles = []

all_files = os.listdir('./output/whitepng')
for f in all_files:
    if f[:4] == 'pial' and f[-4:] == '.png' and '_' not in f:
        pialfiles.append(f)
    if f[:5] == 'white' and f[-4:] == '.png' and '_' not in f:
        whitefiles.append(f)  
        
all_files = os.listdir('./output/pialpng')
for f in all_files:
    if f[:4] == 'pial' and f[-4:] == '.png' and '_' not in f:
        pialfiles.append(f)
    if f[:5] == 'white' and f[-4:] == '.png' and '_' not in f:
        whitefiles.append(f) 

In [6]:
pialfiles

['pialvox2cortex.png',
 'pialpialnn.png',
 'pialcorticalflow.png',
 'pialdeepcsr.png',
 'pialcortexode.png']

In [7]:
whitefiles

['whitevox2cortex.png',
 'whitetopofit.png',
 'whitecortexode.png',
 'whitecorticalflow.png',
 'whitedeepcsr.png']

### Convert PNG to SVG

In [9]:
from png2svglinlin import Png2Svg

In [11]:
# Png2Svg()

### modify SVG file to remove reduplicated ids

In [12]:
import random

#read input file
def modify_svg(svgfile, savetofile):
    fin = open(svgfile, "rt")

    #read file contents to string
    data = fin.read()
    
    #replace all occurrences of the required string
    
    # generate random str
    x = random.random()
    x = int(x*100000)
    data = data.replace('id="img', 'id="img'+str(x))
    data = data.replace('#img', '#img'+str(x))
    data = data.replace('id="clip', 'id="clip'+str(x))
    data = data.replace('#clip', '#clip'+str(x))    
    
    #close the input file
    fin.close()

    #open the input file in write mode
    fin = open('modified_' + svgfile, "wt")

    #overrite the input file with the resulting data
    fin.write(data)

    #close the file
    fin.close()

## **Put meshes In One Figure**

In [23]:
import svgutils.transform as sg
import sys 

#create new SVG figure
figure = sg.SVGFigure("30cm", "15cm")

# load matpotlib-generated figures
prefix = './data-layout-meshes/pial/1-'
# models = models1

start = 150
x0 = 100 + start
xend = 1000 + start
xpos = list(np.linspace(x0,xend,6))

plots = []
txts = []

modelnames = ['CortexODE','CorticalFlow','DeepCSR','PialNN','Vox2Cortex']
scale = .1

for i in range(5):
    fig = sg.fromfile(prefix + str(i+1) + '.svg')
    plot = fig.getroot()
    plot.rotate(90)
    plot.moveto(xpos[i], 0, scale, scale)
    plots.append(plot)
    
prefix = './data-layout-meshes/white/2-'
for i in range(5):
    fig = sg.fromfile(prefix + str(i+1) + '.svg')
    plot = fig.getroot()
    plot.rotate(90)
    plot.moveto(xpos[i], 200, scale, scale)
    plots.append(plot)


sgh = 240
delta = -150

xpos = list(np.linspace(x0,xend+20,6))

txts = []
for i in range(5):
    if i!=3:
        txts.append(sg.TextElement(xpos[i] + delta,sgh, modelnames[i], size=12, weight="bold"))
        
txts.append(sg.TextElement(xpos[3] + delta,sgh-10, modelnames[3]+'*', size=12, weight="bold"))
txts.append(sg.TextElement(xpos[3] + delta,sgh+10, 'Topofit*', size=12, weight="bold"))


rotate = 270

txt1 = sg.TextElement(0, 0, 'Pial', size=12, weight="bold")
txt1.rotate(rotate)
txt1.moveto(50,150)
txt2 = sg.TextElement(0, 0, 'White', size=12, weight="bold")
txt2.rotate(rotate)
txt2.moveto(50,350)

txts.append(txt1)
txts.append(txt2)

# legend

scale = .4
x = 560
y = 440
fig = sg.fromfile('./data-layout-meshes/legend.svg')
plot = fig.getroot()
plot.moveto(x, y,scale,scale)
plots.append(plot)
txts.append(sg.TextElement(x+140 , y - 10 , 'Vertex Distance / Mesh Diag (%)', size=12))

# append plots and labels to figure
figure.append(txts)
figure.append(plots)


# save generated SVG files
figure.save("fig_final03.svg")

## trim images

png need to be scaled smaller to reduce the image size.